In [1]:
import numpy as np
from pandas import DataFrame, read_csv
import pandas as pd 
from scipy.sparse import lil_matrix

file = r'anonymous-msweb.csv'

In [2]:
raw_df = pd.read_csv(file,names = ["category", "value",'other-1','other-2'])
raw_df = raw_df[['category','value']]
raw_df = raw_df[(raw_df.category == 'C') | (raw_df.category == 'V')]

users = raw_df[(raw_df.category == 'C')].value.to_numpy()
items = raw_df[(raw_df.category == 'V')].value.to_numpy()
items = list(set(items))
users = list(set(users))
items.sort()
users.sort()

raw_df['cumsum'] = 0

raw_df.loc[raw_df['category'] == 'C', 'cumsum'] = [1]

raw_df['cumsum'] = raw_df['cumsum'].cumsum()

values = raw_df[(raw_df.category == 'V')]

values['user'] = values['cumsum'].apply(lambda x: users[x-1])
values['item'] = values['value']
values = values[['user', 'item']]

n = len(users)
m = len(items)
sparse_mat =  np.zeros((n,m))

for index, row in values.iterrows():
    sparse_mat[users.index(row['user']), items.index(row['item'])] = 1

items = list(map(lambda x: "X"+str(x), items))
users = list(map(lambda x: str(x), users))
df = pd.DataFrame(sparse_mat, index=users, columns=items)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [3]:
df.to_csv("mat.csv")